In [1]:
import pandas as pd
import sqlalchemy 
import psycopg2
import os
import numpy as np
import requests
import re
import copy
from pandas.api.types import is_numeric_dtype
from sklearn.linear_model import LinearRegression
from sqlalchemy import create_engine
import datetime
from datetime import datetime as dt
import sys
import importlib

In [2]:
sys.path.append('..')

In [3]:
import modules.transforms

In [4]:
importlib.reload(modules.transforms)

<module 'modules.transforms' from '../modules/transforms.py'>

In [5]:
from modules.transforms import *

In [6]:
DBname=os.environ['DB_NAME']
postgres_psswd=os.environ['POSTGRES_PASSWORD']
postgres_user=os.environ['POSTGRES_USER']
postgres_port=str(os.environ['POSTGRES_PORT'])

In [7]:
# A long string that contains the necessary Postgres login information
postgres_str = ('postgresql://'+postgres_user+':'+postgres_psswd+'@'+DBname+':'+postgres_port+'/superset')
# Create the connection
cnx = create_engine(postgres_str)

In [8]:
#guardar paso en csv
path='/data/ETLcache/'
now = dt.now()
timestamp = now.strftime("_%d%m%Y_%H%M%S")

In [9]:
#OBTENER RUTAS DE CARPETAS DISPONIBLES EN REPO COVID DE MINISTERIO DE CIENCIAS DE CHILE
url="https://github.com/MinCiencia/Datos-COVID19/tree/master/output/"
request=requests.get(url)
root=url.split('github.com')[1]
prefix=url.split('/Min')[0]
folders=request.text.split(root)

#product folders
for idx in range(len(folders)):
    f=folders[idx]
    f=f.split('">')[0]
    folders[idx]=f
folders=[prefix+root+f for f in folders if 'producto' in f]

In [10]:
#DEFINIR LISTADO DE PRODUCTOS QUE SE VAN A CONSULTAR DE REPO DE MINISTERIO DE CIENCIAS

productos=['producto5','producto3','producto10','producto13','producto25','producto14','producto19','producto37','producto53','producto54','producto52','producto50','producto48','producto24'
    
]

In [11]:
#LISTADO DE CARPETAS (RUTAS) FILTRADO, SEGÚN PRODUCTOS A CONSULTAR
fs=[]
for p in productos:
    fs.extend([ x for x in folders if p==x.rsplit('/',1)[1]])

In [12]:
#guardar listado de productos consultados en DF (para posteriormente guardar en BD POstgres)
productos=pd.DataFrame(productos)

In [13]:
#crear esquema "tracking" si es que no existe en BD Postgres (para llevar registro de los productos consultados)
k='tracking'
if not cnx.dialect.has_schema(cnx, k):
    print('schema '+k+' does not exist, creating it')
    cnx.execute(sqlalchemy.schema.CreateSchema(k))
else:
    print('schema '+k+' exists, will not be created')

productos.to_sql('min_ciencias_products',con=cnx,schema=k,if_exists='replace')

schema tracking exists, will not be created


In [14]:
#LEER ARCHIVOS DENTRO DE RUTAS CORRESPONDIENTES A PRODUCTOS, GUARDARLOS EN DICCIONARIO "TABLES"
tables={}
for f in fs:
    files_folder=f.rsplit('/')[-1]
    files_folder_prefix='/MinCiencia/Datos-COVID19/blob/master/output/'+files_folder+'/'
    tables[files_folder]={}
    raw_path='https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/'+files_folder+'/'
    req=requests.get(f)
    files=req.text.split(files_folder_prefix)
    for idx in range(len(files)):
        files[idx]=files[idx].split('">')[0]
    files=[raw_path+f for f in files if '.csv' in f]
    for fl in files:
        try:
            print('reading file: '+fl)
            table=pd.read_csv(fl)
            table_name='_'.join(fl.rsplit('/',2)[-2:]).strip('.csv')
            tables[files_folder][table_name]=table
        except Exception as e:
            print(str(e))
            pass

reading file: https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto5/TotalesNacionales.csv
reading file: https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto5/TotalesNacionales_T.csv
reading file: https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto5/TotalesNacionales_std.csv
reading file: https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto3/CasosTotalesCumulativo.csv
reading file: https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto3/CasosTotalesCumulativo_T.csv
reading file: https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto3/CasosTotalesCumulativo_std.csv
reading file: https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto3/TotalesPorRegion.csv
reading file: https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto3/TotalesPorRegion_T.csv
reading file: https:/

In [15]:
#crear "producto11" como caso especial de producto 4
tables['producto11']={}
tables['producto11']['producto4']=pd.read_csv('https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto11/bulk/producto4.csv')
tables['producto11']['producto4'].columns=[x.replace('*','x').replace('%','porcentaje_') for x in tables['producto11']['producto4'].columns]

In [16]:
ortables=copy.deepcopy(tables)

In [17]:
#normalizar tablas transpuestas 
for s in tables.keys():
    for n in tables[s].keys():
        #print(n)
        if '_T' in n[-2:]:
            print('fixing transposed table: '+n)
            tables[s][n]=transpose(tables[s][n])
#y cambiar campos "fecha" (donde haya) a tipo datetime           
for s in tables.keys():
    print('working schema: '+s)
    for n in tables[s].keys():
        if 'fecha' in [str(x).lower() for x in tables[s][n].columns]:
            print('converting to date in table: '+n)
            tables[s][n]=to_datetime(tables[s][n])
        elif 'Fecha' in [str(x).lower() for x in tables[s][n].columns]:
            print('converting to date in table: '+n)
            tables[s][n]=to_datetime(tables[s][n])

            
            
for s in tables.keys():
    print('working schema: '+s)
    for n in tables[s].keys():
        print(tables[s][n].columns)
        if 'fecha' in [str(x) for x in tables[s][n].columns]:
            print('adding reverse date and index in table: '+n)
            tables[s][n]=reverse_idx(tables[s][n],date_field='fecha')
        elif 'Fecha' in [str(x) for x in tables[s][n].columns]:
            print('adding reverse date and index in table: '+n)
            tables[s][n]=reverse_idx(tables[s][n],date_field='Fecha')
            

fixing transposed table: producto5_TotalesNacionales_T
fixing transposed table: producto3_CasosTotalesCumulativo_T
fixing transposed table: producto3_TotalesPorRegion_T
fixing transposed table: producto10_FallecidosEtario_T
fixing transposed table: producto13_CasosNuevosCumulativo_T
fixing transposed table: producto25_CasosActualesPorComuna_T
fixing transposed table: producto14_FallecidosCumulativo_T
fixing transposed table: producto19_CasosActivosPorComuna_T
fixing transposed table: producto37_Defunciones_T
fixing transposed table: producto37_Defunciones_deis_T
fixing transposed table: producto50_DefuncionesDEISPorComuna_T
fixing transposed table: producto50_DefuncionesDEIS_confirmadosPorComuna_T
fixing transposed table: producto50_DefuncionesDEIS_sospechososPorComuna_T
fixing transposed table: producto48_SOCHIMI_T
fixing transposed table: producto24_CamasHospital_Diario_T
working schema: producto5
converting to date in table: producto5_TotalesNacionale
converting to date in table: pr

working schema: producto48
Index(['Codigo region', 'Region', 'Servicio salud', 'Serie', '2020-04-12',
       '2020-04-13', '2020-04-15', '2020-04-16', '2020-04-18', '2020-04-19',
       ...
       '2020-10-21', '2020-10-22', '2020-10-23', '2020-10-24', '2020-10-25',
       '2020-10-26', '2020-10-27', '2020-10-28', '2020-10-29', '2020-10-30'],
      dtype='object', length=187)
Index(['Codigo region', '01', '02', '03', '04', '05', '05.1', '05.2', '06',
       '07',
       ...
       '12.7', '13.42', '13.43', '13.44', '13.45', '13.46', '13.47', '14.7',
       '15.7', '16.7'],
      dtype='object', length=233)
Index(['Fecha', 'Servicio salud', 'Codigo region', 'Region',
       'Camas ocupadas intensivo', 'Camas totales intensivo',
       'Camas ocupadas intermedio', 'Camas totales intermedio',
       'Vmi covid19 confirmados', 'Vmi covid19 sospechosos', 'Vmi ocupados',
       'Vmi totales'],
      dtype='object')
adding reverse date and index in table: producto48_SOCHIMI_std
working schema

In [18]:
#tables['producto5']['producto5_TotalesNacionales_T']['Fecha']=pd.to_datetime(tables['producto5']['producto5_TotalesNacionales_T']['Fecha'])

In [19]:
#calcular índice y fechas invertidos para tablas en que se desea extraer último dato

#to_reverse_idx=[{'producto54':'producto54_r.nacional'},{'producto53':'producto53_confirmados_nacional'},
#                {'producto53':'producto53_confirmados_regionale'}]

#for n in to_reverse_idx:
#    k,i=list(n.items())[0]
#    print('calculating reverse index and date field in table: '+str(k)+' : '+str(i))
#    tables[k][i]=reverse_idx(tables[k][i],date_field='fecha')
    
#to_reverse_idx=[{'producto5':'producto5_TotalesNacionales_T'},
#                {'producto14':'producto14_FallecidosCumulativo_T'},
#                {'producto14':'producto14_FallecidosCumulativo_std'}]

#for n in to_reverse_idx:
#    k,i=list(n.items())[0]
#    print('calculating reverse index and date field in table: '+str(k)+' : '+str(i))
#    tables[k][i]=reverse_idx(tables[k][i],date_field='Fecha')

In [20]:
#agregar variable dummy para región metropolitana
for s in tables.keys():
    for n in tables[s].keys():
        #print(n)
        if 'region' in [x.lower() for x in tables[s][n].columns]:
            print('creando variable dummy para región metropolitana en tabla: '+n)
            tables[s][n]=make_dummies(tables[s][n],fields=['region'],keep_strings=['metropolitana'],value_map={'metropolitana':{0:'Resto de Chile',1:'Región Metropolitana'}})

creando variable dummy para región metropolitana en tabla: producto3_CasosTotalesCumulativo
creando variable dummy para región metropolitana en tabla: producto3_CasosTotalesCumulativo_std
creando variable dummy para región metropolitana en tabla: producto3_TotalesPorRegion
creando variable dummy para región metropolitana en tabla: producto3_TotalesPorRegion_std
creando variable dummy para región metropolitana en tabla: producto13_CasosNuevosCumulativo
creando variable dummy para región metropolitana en tabla: producto13_CasosNuevosCumulativo_std
creando variable dummy para región metropolitana en tabla: producto25_CasosActualesPorComuna
creando variable dummy para región metropolitana en tabla: producto25_CasosActualesPorComuna_std
creando variable dummy para región metropolitana en tabla: producto14_FallecidosCumulativo
creando variable dummy para región metropolitana en tabla: producto14_FallecidosCumulativo_std
creando variable dummy para región metropolitana en tabla: producto19_Ca

In [21]:
for k in tables.keys():
    if not cnx.dialect.has_schema(cnx, k):
        print('schema '+k+' does not exist, creating it')
        cnx.execute(sqlalchemy.schema.CreateSchema(k))
    else:
        print('schema '+k+' exists, will not be created')

schema producto5 exists, will not be created
schema producto3 exists, will not be created
schema producto10 does not exist, creating it
schema producto13 exists, will not be created
schema producto25 exists, will not be created
schema producto14 exists, will not be created
schema producto19 exists, will not be created
schema producto37 exists, will not be created
schema producto53 exists, will not be created
schema producto54 exists, will not be created
schema producto52 exists, will not be created
schema producto50 exists, will not be created
schema producto48 exists, will not be created
schema producto24 exists, will not be created
schema producto11 exists, will not be created


In [22]:
for schema in tables.keys():
    for name in tables[schema].keys():
        try:
            df=tables[schema][name]
            name=name.replace('-','_')
            print("creating table "+name+' ,schema: '+schema)
            df.to_sql(name, schema=schema,con=cnx,if_exists='replace')
            print("saving table"+path+name+timestamp+'.csv in cache')
            df.to_csv(path+name+timestamp+'.csv',encoding='utf-8')
        except Exception as e: 
            print(str(e))
            pass

creating table producto5_TotalesNacionale ,schema: producto5
saving table/data/ETLcache/producto5_TotalesNacionale_28062021_040649.csv in cache
creating table producto5_TotalesNacionales_T ,schema: producto5
saving table/data/ETLcache/producto5_TotalesNacionales_T_28062021_040649.csv in cache
creating table producto5_TotalesNacionales_std ,schema: producto5
saving table/data/ETLcache/producto5_TotalesNacionales_std_28062021_040649.csv in cache
creating table producto3_CasosTotalesCumulativo ,schema: producto3
saving table/data/ETLcache/producto3_CasosTotalesCumulativo_28062021_040649.csv in cache
creating table producto3_CasosTotalesCumulativo_T ,schema: producto3
saving table/data/ETLcache/producto3_CasosTotalesCumulativo_T_28062021_040649.csv in cache
creating table producto3_CasosTotalesCumulativo_std ,schema: producto3
saving table/data/ETLcache/producto3_CasosTotalesCumulativo_std_28062021_040649.csv in cache
creating table producto3_TotalesPorRegion ,schema: producto3
saving tabl

saving table/data/ETLcache/producto50_DefuncionesDEIS_sospechososPorComuna_T_28062021_040649.csv in cache
creating table producto50_DefuncionesDEIS_sospechososPorComuna_std ,schema: producto50
saving table/data/ETLcache/producto50_DefuncionesDEIS_sospechososPorComuna_std_28062021_040649.csv in cache
creating table producto48_SOCHIMI ,schema: producto48
saving table/data/ETLcache/producto48_SOCHIMI_28062021_040649.csv in cache
creating table producto48_SOCHIMI_T ,schema: producto48
saving table/data/ETLcache/producto48_SOCHIMI_T_28062021_040649.csv in cache
creating table producto48_SOCHIMI_std ,schema: producto48
saving table/data/ETLcache/producto48_SOCHIMI_std_28062021_040649.csv in cache
creating table producto24_CamasHospital_Diario ,schema: producto24
saving table/data/ETLcache/producto24_CamasHospital_Diario_28062021_040649.csv in cache
creating table producto24_CamasHospital_Diario_T ,schema: producto24
saving table/data/ETLcache/producto24_CamasHospital_Diario_T_28062021_040649

In [23]:
os.system('jupyter nbconvert --output /home/jovyan/work/ETLdocs/' + 'ETL_covid-chile.html' + ' --to html ' + 
          '/home/jovyan/work/ETL/covid-chile.ipynb')

65280